# Calculus, Solvers, Matrices in SymPy

In [ ]:
from sympy import *
x, y, z = symbols('x y z')

## 1. Calculus

1.1. **Derivatives**  
1.2. **Integrals**  
1.3. **Numerical integration**  
1.4. **Finite differences**

### 1.1. Derivatives

To take derivatives, use the `diff` function.

In [ ]:
diff(sin(x), x)

The derivative of a composite function is calculated by applying the **chain rule**.  
If $f(x)$ and $g(x)$ are differentiable functions, the derivative of the composite function $h(x) = f(g(x))$ is:

$$
h'(x) = f'(g(x)) \cdot g'(x)
$$

In other words, you differentiate the outer function $f$ with respect to the argument $g(x)$ and multiply by the derivative of the inner function $g(x)$ with respect to $x$.

**Example:**  
If $h(x) = \sin(x^2)$, then:
- $f(u) = \sin(u)$ with $u = g(x) = x^2$
- $f'(u) = \cos(u)$
- $g'(x) = 2x$

So:
$$
h'(x) = \cos(x^2) \cdot 2x
$$

In [ ]:
diff(sin(x**2), x)

`diff` can take multiple derivatives at once. To take multiple derivatives, pass the variable as many times as you wish to differentiate, or pass a number after the variable.

In [ ]:
diff(x**4, x, x, x)

In [ ]:
diff(x**4, x, 3)

You can also take derivatives with respect to many variables at once. Just pass each derivative in order, using the same syntax as for single variable derivatives. 

For example, each of the following will compute $\frac{\partial^7}{\partial x\partial y^2\partial z^4} e^{x y z}$.

In [ ]:
expr = exp(x*y*z)

In [ ]:
diff(expr, x, y, y, z, z, z, z)

In [ ]:
diff(expr, x, y, 2, z, 4)

In [ ]:
diff(expr, x, y, y, z, z, z, z) - diff(expr, x, y, 2, z, 4)

`diff` can also be called as a method. The two ways of calling `diff` are exactly the same, and are provided only for convenience.

In [ ]:
expr.diff(x, y, y, z, 4)

To create an unevaluated derivative, use the `Derivative` class. It has the same syntax as `diff`.

In [ ]:
deriv = Derivative(expr, x, y, y, z, 4)
deriv

To evaluate an unevaluated derivative, use the `doit` method.

In [ ]:
deriv.doit()

In [ ]:
deriv.doit() - expr.diff(x, y, y, z, 4)

### 1.2. Integrals

To compute an integral, use the `integrate` function. There are two kinds of integrals, definite and indefinite. 

In [ ]:
integrate(cos(x), x)

Note that SymPy does not include the constant of integration. If you want it, you can add one yourself.

To compute a definite integral, pass the argument `(integration_variable, lower_limit, upper_limit)`.

For example to compute the following: $\int_0^{\infty} e^{-x}\,dx$

In [ ]:
integrate(exp(-x), (x, 0, oo))

As with indefinite integrals, you can pass multiple limit tuples to perform a multiple integral.

For example to compute the following: $\int_{-\infty}^{\infty}\int_{-\infty}^{\infty} e^{- x^{2} - y^{2}}\, dx\, dy$

In [ ]:
integrate(exp(-x**2 - y**2), (x, -oo, oo), (y, -oo, oo))

If `integrate` is unable to compute an integral, it returns an unevaluated `Integral` object.

In [ ]:
expr = integrate(x**x, x)
print(expr)

expr

As with `Derivative`, you can create an unevaluated integral using `Integral`. To later evaluate this integral, call `doit`.

In [ ]:
expr = Integral(log(x)**2, x)
expr

In [ ]:
expr.doit()

### 1.3. Numerical integration

Numeric integration is a method employed in mathematical analysis to estimate the definite integral of a function across a simplified range. 

SymPy not only facilitates symbolic integration but also provides support for numeric integration. It leverages the precision capabilities of the **mpmath** library to enhance the accuracy of numeric integration calculations.

In [ ]:
x = symbols('x')

integral = Integral(sqrt(2)*x, (x, 0, 1))
integral

In [ ]:
integral.evalf()

To compute the integral with a specified precision:

In [ ]:
integral.evalf(50)

In [ ]:
integral.doit()

### 1.4. Finite differences

So far we have looked at expressions with analytic derivatives and primitive functions respectively. But what if we want to have an expression to estimate a derivative of a curve for which we lack a closed form representation, or for which we don’t know the functional values for yet. One approach would be to use a finite difference approach.

The simplest way the differentiate using finite differences is to use the `differentiate_finite` function:

In [ ]:
f, g = symbols('f g', cls=Function)
differentiate_finite(f(x)*g(x))

If you already have a `Derivative` instance, you can use the `as_finite_difference` method to generate approximations of the derivative to arbitrary order:

In [ ]:
f = Function('f')
dfdx = f(x).diff(x)
dfdx.as_finite_difference()

here the first order derivative was approximated around x using a minimum number of points (2 for 1st order derivative) evaluated equidistantly using a step-size of 1. 

We can use arbitrary steps (possibly containing symbolic expressions):

In [ ]:
f = Function('f')
d2fdx2 = f(x).diff(x, 2)
h = Symbol('h')
d2fdx2.as_finite_difference([-3*h,-h,2*h])

You can also use `apply_finite_diff` which takes `order`, `x_list`, `y_list` and `x0` as parameters:

In [ ]:
x_list = [-3, 1, 2]
y_list = symbols('a b c')
apply_finite_diff(1, x_list, y_list, 0)

## 2. Solvers

2.1. **Solving equations algebraically**    
2.2. **Solving differential equations**

Recall that symbolic equations in SymPy are not represented by = or ==, but by `Eq`.

In [ ]:
eq1 = Eq(x, y)
eq1

In [ ]:
eq2 = Eq(x - y, 0)
eq2

### 2.1. Solving equations algebraically

The main function for solving algebraic equations is `solveset`. The syntax for `solveset` is `solveset(equation, variable=None, domain=S.Complexes)` Where equations may be in the form of `Eq` instances or expressions that are assumed to be equal to zero.

When solving a single equation, the output of `solveset` is a FiniteSet or an Interval or ImageSet of the solutions.

In [ ]:
solveset(x**2 - x, x)

In [ ]:
solveset(x - x, x, domain=S.Reals)

In [ ]:
solveset(sin(x) - 1, x, domain=S.Reals)

#### Linear equations

In the `solveset` module, the linear system of equations is solved using `linsolve`.

- List of Equations Form:

In [ ]:
linsolve([x + y + z - 1, x + y + 2*z - 3 ], (x, y, z))

- Augmented Matrix Form:

In [ ]:
linsolve(Matrix(([1, 1, 1, 1], [1, 1, 2, 3])), (x, y, z))

- $A*x = b$ Form

In [ ]:
M = Matrix(((1, 1, 1, 1), (1, 1, 2, 3)))
A = M[:, :-1]
b = M[:, -1]
system = A, b
linsolve(system, x, y, z)

The order of solution corresponds the order of given symbols.

#### Nonlinear equations

In the solveset module, the non linear system of equations is solved using `nonlinsolve`.

- When only real solution is present:

In [ ]:
a, b, c, d = symbols('a, b, c, d', real=True)
nonlinsolve([x*y - 1, x - 2], x, y)

- When only complex solution is present:

In [ ]:
nonlinsolve([x**2 + 1, y**2 + 1], [x, y])

- When both real and complex solution are present:

In [ ]:
system = [x**2 - 2*y**2 -2, x*y - 2]
vars = [x, y]
nonlinsolve(system, vars)

### 2.2. Solving differential equations

To solve differential equations, use `dsolve`. First, create an undefined function by passing `cls=Function` to the `symbols` function.

In [ ]:
f, g = symbols('f g', cls=Function)

f(x)

Derivatives of f(x) are unevaluated.

In [ ]:
f(x).diff(x)

To represent the differential equation $\ddot{f}(x) - 2\dot{f}(x) + f(x) = \sin(x) $, we would thus use

In [ ]:
diffeq = Eq(f(x).diff(x, x) - 2*f(x).diff(x) + f(x), sin(x))
diffeq

To solve the ODE, pass it and the function to solve for to `dsolve`.

In [ ]:
dsolve(diffeq, f(x))

## 3. Matrices

3.1. **Creating matrices**  
3.2. **Basic operations**  
3.3. **Basic Methods**  

### 3.1. Creating matrices

To make a matrix in SymPy, use the `Matrix` object. A matrix is constructed by providing a list of row vectors that make up the matrix. 

For example, to construct the matrix

$
\begin{bmatrix}
1 & -1 \\
3 & 4 \\
0 & 2
\end{bmatrix}
$

In [ ]:
Matrix([[1, -1], [3, 4], [0, 2]])

To make it easy to make column vectors, a list of elements is considered to be a column vector.

In [ ]:
Matrix([1, 2, 3])

### 3.2. Basic Operations

To get the shape of a matrix, use `shape` function.

In [ ]:
M = Matrix([[1, 2, 3], [-2, 0, 4]])

shape(M)

To get an individual row or column of a matrix, use `row` or `col`.

In [ ]:
M.row(0)

In [ ]:
M.col(-1)

### 3.3. Basic Methods

In [ ]:
M = Matrix([[1, 3], [-2, 3]])
N = Matrix([[0, 3], [0, 7]])

As noted above, simple operations like addition, multiplication and power are done just by using `+`, `*`, and `**`.

In [ ]:
M + N

In [ ]:
M*N

In [ ]:
3*M

In [ ]:
M**2

To find the inverse of a matrix, just raise it to the `-1` power.

In [ ]:
M**-1

In [ ]:
N**-1

To take the transpose of a Matrix, use `T`.

In [ ]:
M = Matrix([[1, 2, 3], [4, 5, 6]])
M

In [ ]:
M.T